In [1]:
%matplotlib inline


Voxel-Based Morphometry on Oasis dataset
========================================

This example uses Voxel-Based Morphometry (VBM) to study the relationship
between aging and gray matter density.

The data come from the `OASIS <http://www.oasis-brains.org/>`_ project.
If you use it, you need to agree with the data usage agreement available
on the website.

It has been run through a standard VBM pipeline (using SPM8 and
NewSegment) to create VBM maps, which we study here.

Predictive modeling analysis: VBM bio-markers of aging?
--------------------------------------------------------

We run a standard SVM-ANOVA nilearn pipeline to predict age from the VBM
data. We use only 100 subjects from the OASIS dataset to limit the memory
usage.

Note that for an actual predictive modeling study of aging, the study
should be ran on the full set of subjects. Also, parameters such as the
smoothing applied to the data and the number of features selected by the
Anova step should be set by nested cross-validation, as they impact
significantly the prediction score.

Brain mapping with mass univariate
-----------------------------------

SVM weights are very noisy, partly because heavy smoothing is detrimental
for the prediction here. A standard analysis using mass-univariate GLM
(here permuted to have exact correction for multiple comparisons) gives a
much clearer view of the important regions.

____





In [1]:
# Authors: Elvis Dhomatob, <elvis.dohmatob@inria.fr>, Apr. 2014
#          Virgile Fritsch, <virgile.fritsch@inria.fr>, Apr 2014
#          Gael Varoquaux, Apr 2014
import numpy as np
import matplotlib.pyplot as plt
from nilearn import datasets
from nilearn.input_data import NiftiMasker

n_subjects = 2  # more subjects requires more memory

Load Oasis dataset
-------------------





In [2]:
import os
import numpy as np
import nibabel as nib
import configparser
import json
import csv
from nilearn import plotting
from enum import Enum, auto
import re



# parse in config to see where to read the files in from

DementiaStrings = ["AD dem/FLD prior to AD dem", 
                    "AD dem w/depresss  not contribut", 
                    "AD dem w/depresss- not contribut", 
                    "AD dem w/Frontal lobe/demt at onset", 
                    "Incipient demt PTP", 
                    "AD dem w/oth (list B) not contrib", 
                    "AD dem distrubed social- prior", 
                    "Vascular Demt  primary", 
                    "AD dem Language dysf with", 
                    "AD dem distrubed social- with", 
                    "DAT", 
                    "DLBD- secondary", 
                    "DAT w/depresss not contribut", 
                    "Frontotemporal demt. prim", 
                    "AD dem Language dysf after", 
                    "AD dem w/CVD not contrib", 
                    "AD dem w/oth unusual features", 
                    "AD dem w/PDI after AD dem not contrib", 
                    "AD dem w/oth (list B) contribut", 
                    "AD dem cannot be primary", 
                    "AD dem Language dysf prior", 
                    "AD dem visuospatial- prior", 
                    "AD dem w/oth unusual features/demt on", 
                    "AD dem w/depresss- contribut", 
                    "AD dem w/CVD contribut", 
                    "AD dem visuospatial- with", 
                    "DLBD- primary", 
                    "Incipient Non-AD dem", 
                    "Dementia/PD- primary", 
                    "AD dem distrubed social- after", 
                    "Vascular Demt- secondary", 
                    "AD dem w/depresss  contribut", 
                    "AD Dementia", 
                    "AD dem w/PDI after AD dem contribut", 
                    "AD dem w/oth unusual feat/subs demt", 
                    "Vascular Demt- primary"]
NoDementiaStrings = ["ProAph w/o dement", 
                        "Non AD dem- Other primary", 
                        "Cognitively normal", 
                        "No dementia"]
class Dementia(Enum):
    DEMENTIA = "Dementia"
    NO_DEMENTIA = "No_Dementia"
    UNKNOWN = "Unknown"

    @staticmethod
    def from_str(string):
        if string in DementiaStrings:
            return Dementia.DEMENTIA
        elif string in NoDementiaStrings:
            return Dementia.NO_DEMENTIA
        return Dementia.UNKNOWN


patient_results = {}
with open('../patient_diagnosis.csv') as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader)
    for row in reader:
        rowObj = {}
        for h, v in zip(headers, row):
            rowObj[h] = v
        if rowObj['Subject'] not in patient_results:
            patient_results[rowObj['Subject']] = []
        patient_results[rowObj['Subject']].append(rowObj)

patient_id_to_results = {}
for subject, subjectData in patient_results.items():
    for data in subjectData:
        if data['dx1'] is '':
            continue
        if subject not in patient_id_to_results:
            patient_id_to_results[subject] = [Dementia.from_str(data['dx1'])]
        else:
            patient_id_to_results[subject].append(Dementia.from_str(data['dx1']))


for subject, data in patient_id_to_results.items():
    if len(data) is 1:
        patient_id_to_results[subject] = data[0]
    elif len(data) > 1 and len(set(data)) != 1:
        if Dementia.DEMENTIA in data:
            patient_id_to_results[subject] = Dementia.DEMENTIA
        elif Dementia.NO_DEMENTIA in data:
            patient_id_to_results[subject] = Dementia.NO_DEMENTIA
        else:
            patient_id_to_results[subject] = Dementia.UNKNOWN
    else:
        patient_id_to_results[subject] = data[0]


config = configparser.ConfigParser()
config.read('../config.ini')
location = config['FILES']['Location']

# find all the files in the folder
raw_scan_ids = [x for x in os.listdir(location)]
if '.DS_Store' in raw_scan_ids: raw_scan_ids.remove('.DS_Store')

id_extractor_pattern = re.compile('(.*?)_')

labelled_scan_ids = []

for raw_scan_id in raw_scan_ids:
    m = id_extractor_pattern.match(raw_scan_id)
    scan_id = m.group(1)
    if scan_id not in patient_id_to_results:
        continue
    elif patient_id_to_results[scan_id] is Dementia.UNKNOWN:
        continue
    labelled_scan_ids.append((scan_id, raw_scan_id))

FLAG = "T1w" #or T2w
mri_extractor_pattern = re.compile('anat[0-9]*')

if FLAG is "T1w":
    mri_type_pattern = re.compile(".*T1w.nii.gz")
else:
    mri_type_pattern = re.compile(".*T2w.nii.gz")

paths = {}


for scan_id, raw_scan_id in labelled_scan_ids:
    raw_scan_id = location + "/" + raw_scan_id
    scans = [x for x in os.listdir(raw_scan_id) if mri_extractor_pattern.match(x)]
    scanLocations = []
    for scan in scans:
        scan_folder = raw_scan_id + "/" + scan
        scanTypeMatch = [x for x in os.listdir(scan_folder) if mri_type_pattern.match(x)]
        if scanTypeMatch:
            scanLocations.append(scan_folder + "/" + scanTypeMatch[0])
    if scan_id not in paths:
        paths[scan_id] = []
    paths[scan_id].append(scanLocations)
print(paths)
    

{'OAS30001': [['/Users/RichardMin/CS168/files/OAS30001_MR_d0129/anat3/sub-OAS30001_sess-d0129_run-02_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0129/anat2/sub-OAS30001_sess-d0129_run-01_T1w.nii.gz'], ['/Users/RichardMin/CS168/files/OAS30001_MR_d0757/anat3/sub-OAS30001_sess-d0757_run-02_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0757/anat2/sub-OAS30001_sess-d0757_run-01_T1w.nii.gz']]}


In [3]:
def load_nifti(fname):
        nifti_obj = nib.load(fname)
        nifti_data = nifti_obj.get_data()#.astype(dtype)
        return nifti_data

    

In [4]:

#load oasis dataset 

#oasis_dataset = datasets.paths(n_subjects=n_subjects)

#age = oasis_dataset.ext_vars['age'].astype(float)
#print(gray_matter_map_filenames)
#print(oasis_dataset)
# print basic information on the dataset
#print('First gray-matter anatomy image (3D) is located at: %s' %
 #     oasis_dataset.gray_matter_maps[0])  # 3D data
#print('First white-matter anatomy image (3D) is located at: %s' %
 #     oasis_dataset.white_matter_maps[0])  # 3D data
    
path_string = []

for personId, MRISession in paths.items():

    for MRI in sorted(MRISession):
        for scan in sorted(MRI):
            path_string = path_string + list(MRI) 
        

print(path_string)

for personId, MRISession in paths.items():
    print('asdf')
    for MRI in sorted(MRISession):
        for scan in sorted(MRI):
            img = nib.load(scan)
           # print('Mask nifti image (3D) is located at (Note this is definitely not a "mast nifti image", was playing around with img.get_data(): %s' % img.get_data())
           # print(scan) #this is the actual location of the data
        pass
    
    
filenames=path_string

['/Users/RichardMin/CS168/files/OAS30001_MR_d0129/anat3/sub-OAS30001_sess-d0129_run-02_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0129/anat2/sub-OAS30001_sess-d0129_run-01_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0129/anat3/sub-OAS30001_sess-d0129_run-02_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0129/anat2/sub-OAS30001_sess-d0129_run-01_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0757/anat3/sub-OAS30001_sess-d0757_run-02_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0757/anat2/sub-OAS30001_sess-d0757_run-01_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0757/anat3/sub-OAS30001_sess-d0757_run-02_T1w.nii.gz', '/Users/RichardMin/CS168/files/OAS30001_MR_d0757/anat2/sub-OAS30001_sess-d0757_run-01_T1w.nii.gz']
asdf


In [11]:
#preprocess data
nifti_masker = NiftiMasker(
    standardize=False,
    smoothing_fwhm=2,
    memory='nilearn_cache')  # cache options

transformed_nib_images = []
for filename in filenames:
    image = nib.load(filename)
    print(image.shape)
    print(type(image))
    transformed_nib_images.append(image)
    pass
# gm_maps_masked = nifti_masker.fit_transform(filenames)
# print(gm_maps_masked.shape)
# n_samples, n_features = gm_maps_masked.shape
# print("%d samples, %d features" % (n_subjects, n_features))

(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>
(176, 256, 256)
<class 'nibabel.nifti1.Nifti1Image'>


ValueError: Field of view of image #1 is different from reference FOV.
Reference affine:
array([[  9.96810973e-01,  -1.34100309e-02,   7.87301585e-02,
         -8.90206375e+01],
       [  7.32105831e-03,   9.96994495e-01,   7.71253705e-02,
         -1.08050598e+02],
       [ -7.95282051e-02,  -7.63026252e-02,   9.93908048e-01,
         -1.46887207e+02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])
Image affine:
array([[  9.96803343e-01,  -1.34120397e-02,   7.87297711e-02,
         -8.90189972e+01],
       [  7.32100848e-03,   9.96992528e-01,   7.71507844e-02,
         -1.08053558e+02],
       [ -7.95275569e-02,  -7.63279647e-02,   9.93906140e-01,
         -1.46883850e+02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])
Reference shape:
(176, 256, 256)
Image shape:
(176, 256, 256, 1)
